In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as f
from torch import optim
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
from torchvision.transforms import transforms

In [5]:
t=transforms.Compose([  transforms.ToTensor(),
                        transforms.Normalize((0.5),(0.5)),
                        ])

In [6]:
train_dataset=MNIST(root=r'./data',download=True,transform=t,train=True)
test_dataset=MNIST(root=r'./data',download=True,transform=t,train=False)

In [18]:
train_loader=DataLoader(train_dataset,batch_size=32,shuffle=True)
test_loader=DataLoader(test_dataset,batch_size=32,shuffle=False)

In [7]:
train_dataset.data.shape

torch.Size([60000, 28, 28])

In [ ]:
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet,self).__init__()
        self.fc1=nn.Linear(28*28,256)
        self.fc2=nn.Linear(256,128)
        self.fc3=nn.Linear(128,10)
    def forward(self,x):
        x=x.flatten(1)
        x=x.view(-1,784)
        x=self.fc1(x)
        x=f.relu(x)
        x=self.fc2(x)
        x=f.relu(x)
        x=self.fc3(x)
        return x

In [20]:
model=SimpleNet()
criterian=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=0.0001)


In [22]:
num_epoch=10
for epoch in range(num_epoch):
    running_loss=0.0
    for image,label in train_loader:
        output=model(image)
        loss=criterian(output,label)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        running_loss+=loss.item()
    print(f"epoch[{epoch+1}/{num_epoch}]- loss:{running_loss/len(train_loader)}")

epoch[1/10]- loss:0.5166990224182606
epoch[2/10]- loss:0.2635216014921665
epoch[3/10]- loss:0.205489792684714
epoch[4/10]- loss:0.1657561663935582
epoch[5/10]- loss:0.13823032176444927
epoch[6/10]- loss:0.11745140944421291
epoch[7/10]- loss:0.10081394655555487
epoch[8/10]- loss:0.0876675929085662
epoch[9/10]- loss:0.07659107368613283
epoch[10/10]- loss:0.06813032680414617


In [ ]:
model.eval()
with torch.no_grad():
    correct=0.0
    total=0.0
    for image,label in test_loader:
        output=model(image)
        _,predicted=torch.max(output,1)
        correct+=(predicted==label).sum().item()
        total+=label.size(0) 
    print(f"accuracy: {(correct/total)*100}")

accuracy: 97.37
